# Importing the required libraries

In [17]:
import tkinter as tk
from tkinter import messagebox, filedialog
from tkinter import ttk
import heapq
import math
from collections import Counter

# Huffman encoding

In [18]:
def huffman_compress(data):
    class Node:
        def __init__(self, char, freq):
            self.char = char
            self.freq = freq
            self.left = None
            self.right = None

        def __lt__(self, other):
            return self.freq < other.freq

    def build_tree(frequencies):
        heap = [Node(char, freq) for char, freq in frequencies.items()]
        heapq.heapify(heap)

        while len(heap) > 1:
            left = heapq.heappop(heap)
            right = heapq.heappop(heap)
            merged = Node(None, left.freq + right.freq)
            merged.left = left
            merged.right = right
            heapq.heappush(heap, merged)

        return heap[0]

    def build_codes(node, prefix="", code_map={}):
        if node:
            if node.char is not None:
                code_map[node.char] = prefix
            build_codes(node.left, prefix + "0", code_map)
            build_codes(node.right, prefix + "1", code_map)
        return code_map

    frequencies = Counter(data)
    root = build_tree(frequencies)
    huffman_codes = build_codes(root)
    compressed = ''.join(huffman_codes[char] for char in data)
    return compressed, huffman_codes

# Huffman Decoding

In [19]:
# Huffman Encoding and Decoding
def huffman_compress(data):
    class Node:
        def __init__(self, char, freq):
            self.char = char
            self.freq = freq
            self.left = None
            self.right = None

        def __lt__(self, other):
            return self.freq < other.freq

    def build_tree(frequencies):
        heap = [Node(char, freq) for char, freq in frequencies.items()]
        heapq.heapify(heap)

        while len(heap) > 1:
            left = heapq.heappop(heap)
            right = heapq.heappop(heap)
            merged = Node(None, left.freq + right.freq)
            merged.left = left
            merged.right = right
            heapq.heappush(heap, merged)

        return heap[0]

    def build_codes(node, prefix="", code_map={}):
        if node:
            if node.char is not None:
                code_map[node.char] = prefix
            build_codes(node.left, prefix + "0", code_map)
            build_codes(node.right, prefix + "1", code_map)
        return code_map

    frequencies = Counter(data)
    root = build_tree(frequencies)
    huffman_codes = build_codes(root)
    compressed = ''.join(huffman_codes[char] for char in data)
    return compressed, huffman_codes

def huffman_decompress(data, codes):
    reverse_codes = {v: k for k, v in codes.items()}
    current_code = ""
    decompressed = ""

    for bit in data:
        current_code += bit
        if current_code in reverse_codes:
            decompressed += reverse_codes[current_code]
            current_code = ""

    return decompressed

# RLE Encoding

In [20]:
def rle_compress(data):
    compressed = []
    count = 1
    for i in range(1, len(data)):
        if data[i] == data[i - 1]:
            count += 1
        else:
            compressed.append(f"{data[i - 1]}{count}")
            count = 1
    compressed.append(f"{data[-1]}{count}")
    return ''.join(compressed)

# RLE Decoding

In [21]:
def rle_decompress(data):
    decompressed = []
    char = ""
    count = ""

    for c in data:
        if c.isdigit():
            count += c  
        else:
            if char:  
                decompressed.append(char * int(count) if count else char)
            char = c  
            count = ""  
    if char:
        decompressed.append(char * int(count) if count else char)

    return ''.join(decompressed)

# Golomb Encoding

In [22]:
def golomb_compress(data, m):
    compressed = []
    for number in data:
        q, r = divmod(number, m)
        compressed.append("1" * q + "0" + format(r, f"0{math.ceil(math.log2(m))}b"))
    return ''.join(compressed)

# Golomb Decoding

In [23]:
def golomb_decompress(data, m):
    decompressed = []
    i = 0
    while i < len(data):
        q = 0
        while data[i] == "1":
            q += 1
            i += 1
        i += 1  # Skip the '0'
        r_bits = math.ceil(math.log2(m))
        r = int(data[i:i + r_bits], 2)
        i += r_bits
        decompressed.append(q * m + r)
    return decompressed

# Arithmetic Encoding

In [24]:
def arithmetic_compress(data):
    freq = Counter(data)
    total = sum(freq.values())
    low, high = 0.0, 1.0
    for char in data:
        range_ = high - low
        high = low + range_ * (sum(freq[c] for c in freq if c <= char) / total)
        low = low + range_ * (sum(freq[c] for c in freq if c < char) / total)
    return (low + high) / 2

# Arithmetic Decoding

In [25]:
def arithmetic_decompress(code, data_length, freq):
    total = sum(freq.values())
    low, high = 0.0, 1.0
    result = ""
    for _ in range(data_length):
        range_ = high - low
        value = (code - low) / range_
        cumulative = 0.0
        for char, f in sorted(freq.items()):
            cumulative += f / total
            if value < cumulative:
                result += char
                high = low + range_ * cumulative
                low = low + range_ * (cumulative - f / total)
                break
    return result

# GUI

In [26]:
import tkinter as tk
from tkinter import ttk, messagebox, simpledialog
from collections import Counter

class CompressionApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Data Compression Tool")
        self.root.geometry("600x500")
        self.root.configure(bg="#f2f2f2")

        self.algorithm_var = tk.StringVar(value="Huffman")

        # Styles
        style = ttk.Style()
        style.configure("TLabel", font=("Arial", 12), background="#f2f2f2")
        style.configure("TButton", font=("Arial", 12), padding=5)
        style.configure("TRadiobutton", font=("Arial", 12), background="#f2f2f2")

        # Header
        header = tk.Label(root, text="Data Compression Tool", font=("Arial", 16, "bold"), bg="#4CAF50", fg="white", pady=10)
        header.pack(fill=tk.X)

        # Input Section
        ttk.Label(root, text="Enter Data:").pack(pady=5)
        self.input_text = tk.Text(root, height=5, width=60, font=("Arial", 12))
        self.input_text.pack(pady=5)

        # Input Instructions
        self.instructions_label = ttk.Label(root, text="Input Instructions: Enter text data.")
        self.instructions_label.pack(pady=5)

        # Algorithm Selection
        ttk.Label(root, text="Select Algorithm:").pack(pady=5)
        algorithms = ["Huffman", "Golomb", "RLE", "Arithmetic"]
        for algo in algorithms:
            ttk.Radiobutton(root, text=algo, variable=self.algorithm_var, value=algo, command=self.update_instructions).pack(anchor=tk.W, padx=20)

        # Buttons
        button_frame = tk.Frame(root, bg="#f2f2f2")
        button_frame.pack(pady=10)
        ttk.Button(button_frame, text="Compress", command=self.compress).pack(side=tk.LEFT, padx=10)
        ttk.Button(button_frame, text="Decompress", command=self.decompress).pack(side=tk.LEFT, padx=10)

        # Output Section
        ttk.Label(root, text="Output:").pack(pady=5)
        self.output_text = tk.Text(root, height=5, width=60, font=("Arial", 12), state=tk.DISABLED)
        self.output_text.pack(pady=5)

    def update_instructions(self):
        """Update input instructions based on the selected algorithm."""
        algorithm = self.algorithm_var.get()
        if algorithm == "Huffman":
            instructions = "Input Instructions: Enter any text data to compress using Huffman encoding."
        elif algorithm == "RLE":
            instructions = "Input Instructions: Enter text data with repetitive patterns for Run-Length Encoding (RLE)."
        elif algorithm == "Golomb":
            instructions = (
                "Input Instructions: Enter space-separated integers (e.g., '5 9 12 3').\n"
                "For compression, you will be prompted to enter a divisor (m)."
            )
        elif algorithm == "Arithmetic":
            instructions = "Input Instructions: Enter any text data to compress using Arithmetic encoding."
        else:
            instructions = "Input Instructions: Unknown algorithm."

        self.instructions_label.config(text=instructions)

    def compress(self):
        data = self.input_text.get("1.0", tk.END).strip()
        if not data:
            messagebox.showerror("Error", "Input data cannot be empty.")
            return

        algorithm = self.algorithm_var.get()
        if algorithm == "Huffman":
            compressed, codes = huffman_compress(data)
            self.codes = codes  # Save for decompression
            self.display_output(f"Compressed Data: {compressed}\nCodes: {codes}")
        elif algorithm == "RLE":
            compressed = rle_compress(data)
            self.display_output(f"Compressed Data: {compressed}")
        elif algorithm == "Golomb":
            try:
                numbers = list(map(int, data.split()))
                divisor = simpledialog.askinteger("Golomb Compression", "Enter the divisor (m):")
                if divisor is None or divisor <= 0:
                    messagebox.showerror("Error", "Invalid divisor. It must be a positive integer.")
                    return
                compressed = golomb_compress(numbers, divisor)
                self.m = divisor  # Save for decompression
                self.display_output(f"Compressed Data: {compressed}")
            except ValueError:
                messagebox.showerror("Error", "For Golomb coding, enter space-separated integers.")
        elif algorithm == "Arithmetic":
            freq = Counter(data)
            compressed = arithmetic_compress(data)
            self.freq = freq  # Save for decompression
            self.data_length = len(data)  # Save for decompression
            self.display_output(f"Compressed Data: {compressed}")

    def decompress(self):
        compressed_data = self.input_text.get("1.0", tk.END).strip()
        if not compressed_data:
            messagebox.showerror("Error", "Input data cannot be empty.")
            return

        algorithm = self.algorithm_var.get()
        if algorithm == "Huffman":
            if not hasattr(self, "codes"):
                messagebox.showerror("Error", "No Huffman codes found for decompression.")
                return
            decompressed = huffman_decompress(compressed_data, self.codes)
            self.display_output(f"Decompressed Data: {decompressed}")
        elif algorithm == "RLE":
            decompressed = rle_decompress(compressed_data)
            self.display_output(f"Decompressed Data: {decompressed}")
        elif algorithm == "Golomb":
            try:
                divisor = simpledialog.askinteger("Golomb Decompression", "Enter the divisor (m) used for encoding:")
                if divisor is None or divisor <= 0:
                    messagebox.showerror("Error", "Invalid divisor. It must be a positive integer.")
                    return
                decompressed = golomb_decompress(compressed_data, divisor)
                self.display_output(f"Decompressed Data: {decompressed}")
            except ValueError:
                messagebox.showerror("Error", "Invalid data format for Golomb decompression.")
        elif algorithm == "Arithmetic":
            if not hasattr(self, "freq") or not hasattr(self, "data_length"):
                messagebox.showerror("Error", "No frequency table or data length found for Arithmetic decompression.")
                return
            try:
                code = float(compressed_data)
                decompressed = arithmetic_decompress(code, self.data_length, self.freq)
                self.display_output(f"Decompressed Data: {decompressed}")
            except ValueError:
                messagebox.showerror("Error", "Invalid data format for Arithmetic decompression.")

    def display_output(self, text):
        """Display output in the output_text box."""
        self.output_text.config(state=tk.NORMAL)
        self.output_text.delete(1.0, tk.END)
        self.output_text.insert(tk.END, text)
        self.output_text.config(state=tk.DISABLED)


if __name__ == "__main__":
    root = tk.Tk()
    app = CompressionApp(root)
    root.mainloop()

